In [26]:
# autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

In [28]:
mimic_iv_path = "/data/wang/junh/datasets/physionet.org/files/mimiciv/2.2"
mm_dir = "/data/wang/junh/datasets/multimodal"

output_dir = os.path.join(mm_dir, "preprocessing")

In [29]:
restrict_48_hours = True
include_notes = True
include_cxr = True
include_ecg = True
standard_scale = True
include_missing = False

In [30]:
ireg_vitals_ts_df = pd.read_pickle(os.path.join(output_dir, "ts_labs_vitals_new.pkl"))
ireg_vitals_ts_df = ireg_vitals_ts_df.drop(columns=["hosp_time_delta"])
ireg_vitals_ts_df.rename(columns={'icu_time_delta': 'timedelta'}, inplace=True)
imputed_vitals = pd.read_pickle(os.path.join(output_dir, "imputed_ts_labs_vitals_new.pkl"))

# ireg_vitals_ts_df = pd.read_pickle(os.path.join(output_dir, "ts_labs_vitals_icu.pkl"))
# imputed_vitals = pd.read_pickle(os.path.join(output_dir, "imputed_ts_labs_vitals_icu.pkl"))
print(ireg_vitals_ts_df.columns)
print(imputed_vitals.columns)

Index(['subject_id', 'hadm_id', 'stay_id', 'timedelta', 'Anion Gap',
       'Bicarbonate', 'Calcium, Total', 'Chloride', 'Creatinine',
       'Diastolic BP', 'GCS - Eye Opening', 'GCS - Motor Response',
       'GCS - Verbal Response', 'Glucose', 'Heart Rate', 'Hematocrit',
       'Hemoglobin', 'MCH', 'MCHC', 'MCV', 'Magnesium', 'Mean BP',
       'Neutrophils', 'O2 Saturation', 'Phosphate', 'Platelet Count', 'RDW',
       'Red Blood Cells', 'Respiratory Rate', 'Sodium', 'Systolic BP',
       'Urea Nitrogen', 'Vancomycin', 'White Blood Cells'],
      dtype='object')
Index(['subject_id', 'hadm_id', 'stay_id', 'timedelta', 'Anion Gap',
       'Bicarbonate', 'Calcium, Total', 'Chloride', 'Creatinine',
       'Diastolic BP', 'GCS - Eye Opening', 'GCS - Motor Response',
       'GCS - Verbal Response', 'Glucose', 'Heart Rate', 'Hematocrit',
       'Hemoglobin', 'MCH', 'MCHC', 'MCV', 'Magnesium', 'Mean BP',
       'Neutrophils', 'O2 Saturation', 'Phosphate', 'Platelet Count', 'RDW',
       'Red

In [31]:
print(len(ireg_vitals_ts_df.columns))

34


In [32]:

ireg_vitals_ts_df = ireg_vitals_ts_df[ireg_vitals_ts_df['timedelta'] >= 0]
imputed_vitals = imputed_vitals[imputed_vitals['timedelta'] >= 0]

if restrict_48_hours:
    ireg_vitals_ts_df = ireg_vitals_ts_df[ireg_vitals_ts_df['timedelta'] <= 48]
    imputed_vitals = imputed_vitals[imputed_vitals['timedelta'] <= 48]

In [33]:
if include_notes:
    notes_df = pd.read_pickle(os.path.join(output_dir, "icu_notes_text_embeddings.pkl"))
    print(notes_df.columns)
    # notes_df = pd.read_pickle(os.path.join(output_dir, "notes_text.pkl"))
    notes_df = notes_df[notes_df['stay_id'].notnull()]

    notes_df = notes_df[notes_df['icu_time_delta'] >= 0]
    if restrict_48_hours:
        notes_df = notes_df[notes_df['icu_time_delta'] <= 48]

if include_cxr:
    cxr_df = pd.read_pickle(os.path.join(output_dir, "cxr_embeddings_stay.pkl"))
    print(cxr_df.columns)
    cxr_df = cxr_df[cxr_df['icu_time_delta'] >= 0]
    if restrict_48_hours:
        cxr_df = cxr_df[cxr_df['icu_time_delta'] <= 48]

if include_ecg:
    ecg_df = pd.read_pickle(os.path.join(output_dir, "ecg_embeddings_icu.pkl"))
    ecg_df = ecg_df[ecg_df['icu_time_delta'] >= 0]
    if restrict_48_hours:
        ecg_df = ecg_df[ecg_df['icu_time_delta'] <= 48]

Index(['note_id', 'subject_id', 'hadm_id', 'note_type', 'note_seq',
       'charttime', 'storetime', 'text', 'stay_id', 'icu_time_delta',
       'hosp_time_delta', 'biobert_embeddings'],
      dtype='object')
Index(['dicom_id', 'subject_id', 'study_id',
       'PerformedProcedureStepDescription', 'ViewPosition',
       'ProcedureCodeSequence_CodeMeaning', 'ViewCodeSequence_CodeMeaning',
       'PatientOrientationCodeSequence_CodeMeaning', 'densefeatures',
       'predictions', 'cxrtime', 'hadm_id', 'stay_id', 'icu_time_delta',
       'hosp_time_delta'],
      dtype='object')


In [34]:
cxr_df.head()

,dicom_id,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning,densefeatures,predictions,cxrtime,hadm_id,stay_id,icu_time_delta,hosp_time_delta
72,7b25b3ed-e780a527-319cb7b3-02d5d071-f1cddee9,10001884,50712381,CHEST (PORTABLE AP),AP,CHEST (PORTABLE AP),antero-posterior,Erect,"[0.004360755, 0.10861953, 0.0656413, 0.0077615...","[0.83476615, 0.6821718, 0.5, 0.5015653, 0.7613...",2131-01-12 04:56:56,26184834,37510196,24.614167,104.298889
150,38f6981f-10343eb8-7c974e21-458d5218-f29f1617,10002428,50862960,CHEST (PORTABLE AP),AP,CHEST (PORTABLE AP),antero-posterior,NaN,"[0.0026520048, 0.021410752, 0.23664735, 0.0120...","[0.77319604, 0.6379471, 0.5, 0.2651843, 0.8441...",2156-04-21 02:24:51,28662225,38875437,32.225556,204.1475
151,4e39ddbc-47422ff9-3de977e6-924e3c52-4cac3943,10002428,51574899,CHEST (PORTABLE AP),AP,CHEST (PORTABLE AP),antero-posterior,NaN,"[0.0007566094, 0.029454479, 0.20461331, 0.0143...","[0.9089313, 0.8434784, 0.5, 0.5104989, 0.76865...",2156-04-21 11:13:19,28662225,38875437,41.033333,212.955278
152,54c2ed5c-f4fbc20d-3bf4c783-283c3878-e9eb320d,10002428,52460896,CHEST (PORTABLE AP),AP,CHEST (PORTABLE AP),antero-posterior,NaN,"[0.0022582188, 0.04897062, 0.21259578, 0.02378...","[0.8172648, 0.7462012, 0.5, 0.37802082, 0.7915...",2156-05-12 04:07:00,23473524,35479615,13.290556,13.3
160,217e6a56-b4a78227-a2acc34d-3b571d4a-27b9746a,10002428,57064083,CHEST (PORTABLE AP),AP,CHEST (PORTABLE AP),antero-posterior,NaN,"[0.0008703213, 0.06460024, 0.2580139, 0.002456...","[0.8643331, 0.8319758, 0.5, 0.33241627, 0.8501...",2156-05-11 18:46:22,23473524,35479615,3.946667,3.956111


In [35]:
icustays_df = pd.read_csv(os.path.join(mimic_iv_path, "icu", "icustays.csv.gz"), low_memory=False)
icustays_df['intime'] = pd.to_datetime(icustays_df['intime'])
icustays_df['outtime'] = pd.to_datetime(icustays_df['outtime'])

if restrict_48_hours:
    icustays_df = icustays_df[icustays_df['los'] >= 2]

In [36]:
valid_stay_ids = icustays_df['stay_id'].unique()

ireg_vitals_ts_df = ireg_vitals_ts_df[ireg_vitals_ts_df['stay_id'].isin(valid_stay_ids)]
imputed_vitals = imputed_vitals[imputed_vitals['stay_id'].isin(valid_stay_ids)]

if include_notes:
    notes_df = notes_df[notes_df['stay_id'].isin(valid_stay_ids)]

if include_cxr:
    cxr_df = cxr_df[cxr_df['stay_id'].isin(valid_stay_ids)]

if include_ecg:
    ecg_df = ecg_df[ecg_df['stay_id'].isin(valid_stay_ids)]

In [37]:
ireg_vitals_ts_df.head()

,subject_id,hadm_id,stay_id,timedelta,Anion Gap,Bicarbonate,"Calcium, Total",Chloride,Creatinine,Diastolic BP,...,Phosphate,Platelet Count,RDW,Red Blood Cells,Respiratory Rate,Sodium,Systolic BP,Urea Nitrogen,Vancomycin,White Blood Cells
2523449,10001884,26184834.0,37510196,0.031944,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2523450,10001884,26184834.0,37510196,0.081944,NaN,NaN,NaN,NaN,NaN,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,180.0,NaN,NaN,NaN
2523451,10001884,26184834.0,37510196,0.665278,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN
2523452,10001884,26184834.0,37510196,0.681944,NaN,NaN,NaN,NaN,NaN,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,167.0,NaN,NaN,NaN
2523453,10001884,26184834.0,37510196,1.665278,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,20.0,NaN,NaN,NaN,NaN,NaN


In [38]:
#print("Number of stays in ecg: ", ecg_df['stay_id'].nunique())
print("Number of stays in cxr: ", cxr_df['stay_id'].nunique())
print("Number of stays in notes: ", notes_df['stay_id'].nunique())

Number of stays in cxr:  8781
Number of stays in notes:  32040


In [39]:
print(ireg_vitals_ts_df.shape)

(3014705, 34)


In [40]:
admissions_df = pd.read_csv(os.path.join(mimic_iv_path, "hosp", "admissions.csv.gz"))
admissions_df = admissions_df.rename(columns={"hospital_expire_flag": "died"})
admissions_df = admissions_df[["subject_id", "hadm_id", "died"]]

In [41]:

if not include_missing:
    unique_stays = ireg_vitals_ts_df['stay_id'].unique()
    print(f"Number of stays with vitals: {len(unique_stays)}")

    if include_notes:
        unique_stays = np.intersect1d(unique_stays, notes_df['stay_id'].unique())
        print(f"Number of stays with notes: {len(unique_stays)}")

    if include_cxr:
        unique_stays = np.intersect1d(unique_stays, cxr_df['stay_id'].unique())
        print(f"Number of stays with cxr: {len(unique_stays)}")

    if include_ecg:
        unique_stays = np.intersect1d(unique_stays, ecg_df['stay_id'].unique())
        print(f"Number of stays with ecg: {len(unique_stays)}")
else:
    unique_stays = ireg_vitals_ts_df['stay_id'].unique()
    print(f"Number of stays with vitals: {len(unique_stays)}")

    if include_notes:
        # Get stays with either TS or notes
        unique_stays = np.union1d(unique_stays, notes_df['stay_id'].unique())
        print(f"Number of stays with either TS or notes: {len(unique_stays)}")
    
    if include_cxr:
        unique_stays = np.union1d(unique_stays, cxr_df['stay_id'].unique())
        print(f"Number of stays with either TS, notes, cxr: {len(unique_stays)}")
    
    if include_ecg:
        unique_stays = np.union1d(unique_stays, ecg_df['stay_id'].unique())
        print(f"Number of stays with either TS, notes, cxr, ecg: {len(unique_stays)}")

Number of stays with vitals: 35131
Number of stays with notes: 32040
Number of stays with cxr: 8770
Number of stays with ecg: 5178


In [42]:
stays_with_labels = []
stay_to_hadm_mapping = dict(zip(ireg_vitals_ts_df['stay_id'], ireg_vitals_ts_df['hadm_id']))

for stay in unique_stays:
    curr_stay_dict = {}
    curr_stay_dict['stay_id'] = stay

    # Retrieve hadm_id using the mapping
    if stay in stay_to_hadm_mapping:
        curr_stay_dict['hadm_id'] = stay_to_hadm_mapping[stay]
        # Retrieve the label (e.g., 'died') from admissions_df
        curr_stay_dict['label'] = admissions_df[
            admissions_df['hadm_id'] == curr_stay_dict['hadm_id']
        ]['died'].iloc[0]

    stays_with_labels.append(curr_stay_dict)

stays_df = pd.DataFrame(stays_with_labels)

In [43]:
# # Create train, val, test splits
# np.random.seed(0)
# np.random.shuffle(unique_stays)
# train_stays = unique_stays[:int(0.7*len(unique_stays))]
# val_stays = unique_stays[int(0.7*len(unique_stays)):int(0.85*len(unique_stays))]
# test_stays = unique_stays[int(0.85*len(unique_stays)):]

In [44]:
groups = stays_df.groupby('label')
train_stays, val_stays, test_stays = [], [], []

np.random.seed(0)  # Ensure reproducibility

for label, group in groups:
    shuffled_group = group.sample(frac=1, random_state=0)  # Shuffle
    n = len(shuffled_group)
    
    # Calculate split indices
    train_end = int(0.7 * n)
    val_end = int(0.85 * n)
    
    # Append stratified splits
    train_stays.extend(shuffled_group.iloc[:train_end].to_dict('records'))
    val_stays.extend(shuffled_group.iloc[train_end:val_end].to_dict('records'))
    test_stays.extend(shuffled_group.iloc[val_end:].to_dict('records'))

In [45]:
# Convert train_stays, val_stays, and test_stays dictionaries back to unique_stays format
train_stays = [stay['stay_id'] for stay in train_stays]
val_stays = [stay['stay_id'] for stay in val_stays]
test_stays = [stay['stay_id'] for stay in test_stays]

In [46]:
train_ireg_ts_df = ireg_vitals_ts_df[ireg_vitals_ts_df['stay_id'].isin(train_stays)].copy()
train_imputed_df = imputed_vitals[imputed_vitals['stay_id'].isin(train_stays)].copy()

cols = train_ireg_ts_df.columns.tolist()
cols = [col for col in cols if col not in ['subject_id', 'hadm_id', 'stay_id', 'timedelta']]

if standard_scale:
    for col in cols:
        scaler = StandardScaler()
        scaler.fit(train_ireg_ts_df[[col]])
        ireg_vitals_ts_df[col] = scaler.transform(ireg_vitals_ts_df[[col]])

        scaler = StandardScaler()
        scaler.fit(train_imputed_df[[col]])
        imputed_vitals[col] = scaler.transform(imputed_vitals[[col]])


In [47]:
print(train_ireg_ts_df.columns)

Index(['subject_id', 'hadm_id', 'stay_id', 'timedelta', 'Anion Gap',
       'Bicarbonate', 'Calcium, Total', 'Chloride', 'Creatinine',
       'Diastolic BP', 'GCS - Eye Opening', 'GCS - Motor Response',
       'GCS - Verbal Response', 'Glucose', 'Heart Rate', 'Hematocrit',
       'Hemoglobin', 'MCH', 'MCHC', 'MCV', 'Magnesium', 'Mean BP',
       'Neutrophils', 'O2 Saturation', 'Phosphate', 'Platelet Count', 'RDW',
       'Red Blood Cells', 'Respiratory Rate', 'Sodium', 'Systolic BP',
       'Urea Nitrogen', 'Vancomycin', 'White Blood Cells'],
      dtype='object')


In [52]:
def get_stay_list(stays):
    stays_list = []

    for curr_stay in tqdm(stays):
        curr_stay_ireg = ireg_vitals_ts_df[ireg_vitals_ts_df['stay_id'] == curr_stay].copy()
        curr_stay_imputed = imputed_vitals[imputed_vitals['stay_id'] == curr_stay].copy()

        if len(curr_stay_ireg) == 0:
            continue

        curr_stay_dict = {}
        curr_stay_dict['name'] = curr_stay_ireg['subject_id'].iloc[0]
        curr_stay_dict['hadm_id'] = curr_stay_ireg['hadm_id'].iloc[0]
        curr_stay_dict['stay_id'] = curr_stay
        curr_stay_dict['ts_tt'] = curr_stay_ireg['timedelta'].values

        # Save the feature names before dropping unnecessary columns
        feature_names = curr_stay_ireg.drop(columns=['subject_id', 'hadm_id', 'stay_id', 'timedelta']).columns.tolist()
        curr_stay_ireg.drop(columns=['subject_id', 'hadm_id', 'stay_id', 'timedelta'], inplace=True)
        ireg_ts_mask = curr_stay_ireg.notnull()
        curr_stay_ireg.fillna(0, inplace=True)
        curr_stay_dict['irg_ts'] = curr_stay_ireg.values
        curr_stay_dict['irg_ts_mask'] = ireg_ts_mask.values.astype(int)

        curr_stay_imputed.drop(columns=['subject_id', 'hadm_id', 'stay_id', 'timedelta'], inplace=True)
        curr_stay_dict['reg_ts'] = curr_stay_imputed.values

        if include_notes:
            curr_stay_notes = notes_df[notes_df['stay_id'] == curr_stay].copy()
            if len(curr_stay_notes) == 0:
                curr_stay_dict['text_data'] = []
                curr_stay_dict['text_time_to_end'] = []
                curr_stay_dict['text_embeddings'] = []
                curr_stay_dict['text_missing'] = 1
            else:
                curr_stay_dict['text_data'] = curr_stay_notes['text'].tolist()
                curr_stay_dict['text_time_to_end'] = curr_stay_notes['icu_time_delta'].values
                curr_stay_dict['text_embeddings'] = [emb[0][0] for emb in curr_stay_notes['biobert_embeddings']]
                curr_stay_dict['text_missing'] = 0

        if include_cxr:
            curr_stay_cxr = cxr_df[cxr_df['stay_id'] == curr_stay].copy()
            if curr_stay_cxr.empty:
                curr_stay_dict['cxr_metadata'] = []
                curr_stay_dict['image_paths'] = []
                curr_stay_dict['cxr_missing'] = 1
            else:
                curr_stay_dict['cxr_feats'] = curr_stay_cxr['densefeatures'].tolist()
                metadata = curr_stay_cxr[['dicom_id', 'PerformedProcedureStepDescription', 'ViewPosition', 'ProcedureCodeSequence_CodeMeaning', 'ViewCodeSequence_CodeMeaning', 'PatientOrientationCodeSequence_CodeMeaning']].to_dict('records')
                curr_stay_dict['cxr_metadata'] = metadata
                image_paths = curr_stay_cxr.apply(lambda row: os.path.join('/data/wang/junh/datasets/physionet.org/files/mimic-cxr-jpg/2.0.0/mimic-cxr-jpg-2.1.0.physionet.org/files', f"p{str(row['subject_id'])[:2]}", f"p{row['subject_id']}", f"s{row['study_id']}", f"{row['dicom_id']}.jpg"), axis=1).tolist()
                curr_stay_dict['image_paths'] = image_paths
                curr_stay_dict['cxr_time'] = curr_stay_cxr['icu_time_delta'].values
                curr_stay_dict['cxr_missing'] = 0

        if include_ecg and ecg_df is not None:
            curr_stay_ecg = ecg_df[ecg_df['stay_id'] == curr_stay].copy()
            if len(curr_stay_ecg) == 0:
                curr_stay_dict['ecg_feats'] = []
                curr_stay_dict['ecg_time'] = []
                curr_stay_dict['ecg_missing'] = 1
            else:
                curr_stay_dict['ecg_feats'] = curr_stay_ecg['embeddings'].tolist()
                curr_stay_dict['ecg_time'] = curr_stay_ecg['icu_time_delta'].values
                curr_stay_dict['ecg_missing'] = 0

        if admissions_df is not None:
            curr_stay_dict['label'] = admissions_df[admissions_df['hadm_id'] == curr_stay_dict['hadm_id']]['died'].iloc[0]

        stays_list.append(curr_stay_dict)

    return stays_list


In [49]:
# def get_stay_list(stays):
#     stays_list = []

#     for curr_stay in tqdm(stays):
        
#         curr_stay_ireg = ireg_vitals_ts_df[ireg_vitals_ts_df['stay_id'] == curr_stay].copy()
#         print(curr_stay_ireg)
#         #print(f"Initial irregular time series shape (before dropping columns): {curr_stay_ireg.shape}")
#         curr_stay_imputed = imputed_vitals[imputed_vitals['stay_id'] == curr_stay].copy()

#         if len(curr_stay_ireg) == 0:
#             continue

#         curr_stay_dict = {}
#         curr_stay_dict['name'] = curr_stay_ireg['subject_id'].iloc[0]
#         curr_stay_dict['hadm_id'] = curr_stay_ireg['hadm_id'].iloc[0]
#         curr_stay_dict['stay_id'] = curr_stay
#         curr_stay_dict['ts_tt'] = curr_stay_ireg['timedelta'].values

#         curr_stay_ireg.drop(columns=['subject_id', 'hadm_id', 'stay_id', 'timedelta'], inplace=True)
#         ireg_ts_mask = curr_stay_ireg.notnull()
#         curr_stay_ireg.fillna(0, inplace=True)
#         curr_stay_dict['irg_ts'] = curr_stay_ireg.values
#         curr_stay_dict['irg_ts_mask'] = ireg_ts_mask.values.astype(int)

#         curr_stay_imputed.drop(columns=['subject_id', 'hadm_id', 'stay_id', 'timedelta'], inplace=True)
#         curr_stay_dict['reg_ts'] = curr_stay_imputed.values

#         if include_notes:
#             curr_stay_notes = notes_df[notes_df['stay_id'] == curr_stay].copy()

#             if len(curr_stay_notes) == 0:
#                 curr_stay_dict['text_data'] = []
#                 curr_stay_dict['text_time_to_end'] = []
#                 curr_stay_dict['text_embeddings'] = []
#                 curr_stay_dict['text_missing'] = 1
#             else:
#                 curr_stay_dict['text_data'] = curr_stay_notes['text'].tolist()
#                 curr_stay_dict['text_time_to_end'] = curr_stay_notes['icu_time_delta'].values
#                 # a list of the first [CLS] embedding vectors 
#                 # of the first chunk for each note in curr_stay_notes
#                 curr_stay_dict['text_embeddings'] = [emb[0][0] for emb in curr_stay_notes['biobert_embeddings']]
#                 curr_stay_dict['text_missing'] = 0

#         if include_cxr:
#             curr_stay_cxr = cxr_df[cxr_df['stay_id'] == curr_stay].copy()
            
#             if len(curr_stay_cxr) == 0:
#                 curr_stay_dict['cxr_feats'] = []
#                 curr_stay_dict['cxr_time'] = []
#                 curr_stay_dict['cxr_missing'] = 1
#             else:
#                 curr_stay_dict['cxr_feats'] = curr_stay_cxr['densefeatures'].tolist()
#                 curr_stay_dict['cxr_time'] = curr_stay_cxr['icu_time_delta'].values
#                 curr_stay_dict['cxr_missing'] = 0
#                 #print(f"Number of CXR features: {len(curr_stay_dict['cxr_feats'])}")

#         if include_ecg:
#             curr_stay_ecg = ecg_df[ecg_df['stay_id'] == curr_stay].copy()
#             if len(curr_stay_ecg) == 0:
#                 curr_stay_dict['ecg_feats'] = []
#                 curr_stay_dict['ecg_time'] = []
#                 curr_stay_dict['ecg_missing'] = 1
#             else:
#                 curr_stay_dict['ecg_feats'] = curr_stay_ecg['embeddings'].tolist()
#                 curr_stay_dict['ecg_time'] = curr_stay_ecg['icu_time_delta'].values
#                 curr_stay_dict['ecg_missing'] = 0

#         curr_stay_dict['label'] = admissions_df[admissions_df['hadm_id'] == curr_stay_dict['hadm_id']]['died'].iloc[0]

#         stays_list.append(curr_stay_dict)

#     return stays_list



In [53]:
train_stays_list = get_stay_list(train_stays)
val_stays_list = get_stay_list(val_stays)
test_stays_list = get_stay_list(test_stays)

100%|██████████| 778/778 [01:54<00:00,  6.79it/s]


In [27]:
# Example to check the first patient stay
first_stay = train_stays[18]
first_stay_data = get_stay_list([first_stay])


100%|██████████| 1/1 [00:00<00:00,  7.16it/s]


In [54]:
# Save the data
import pickle

output_dir = os.path.join(mm_dir, "balanced")

base_name = "ihm"
if restrict_48_hours:
    base_name += "-48"
else:
    base_name += "-all"

if include_cxr:
    if include_notes:
        base_name += "-cxr-notes"
    else:
        base_name += "-cxr"

if include_ecg:
    base_name += "-ecg"

if include_missing:
    base_name += "-missingInd"

f_path = os.path.join(output_dir, f"train_{base_name}_stays.pkl")
with open(f_path, 'wb') as f:
    print(f"Saving train stays to {f_path}")
    pickle.dump(train_stays_list, f)

f_path = os.path.join(output_dir, f"val_{base_name}_stays.pkl")
with open(f_path, 'wb') as f:
    print(f"Saving val stays to {f_path}")
    pickle.dump(val_stays_list, f)

f_path = os.path.join(output_dir, f"test_{base_name}_stays.pkl")
with open(f_path, 'wb') as f:
    print(f"Saving test stays to {f_path}")
    pickle.dump(test_stays_list, f)


Saving train stays to /data/wang/junh/datasets/multimodal/balanced/train_ihm-48-cxr-notes-ecg_stays.pkl
Saving val stays to /data/wang/junh/datasets/multimodal/balanced/val_ihm-48-cxr-notes-ecg_stays.pkl
Saving test stays to /data/wang/junh/datasets/multimodal/balanced/test_ihm-48-cxr-notes-ecg_stays.pkl


In [68]:
# Save the data
import pickle
file_path = '/data/wang/junh/datasets/multimodal/augmentation/train_ihm-48-cxr-notes_stays.pkl'
with open(file_path, 'rb') as f:
    train_stays_list = pickle.load(f)
print(len(train_stays_list))

6139


In [69]:
print(train_stays_list[0].keys())

dict_keys(['name', 'hadm_id', 'stay_id', 'ts_tt', 'ts_feature_names', 'irg_ts', 'irg_ts_mask', 'reg_ts', 'text_data', 'text_time_to_end', 'text_missing', 'cxr_metadata', 'image_paths', 'cxr_missing', 'label'])


In [73]:
print(train_stays_list[0].get('text_data'))

['EXAMINATION:  CHEST (PORTABLE AP)\n\nINDICATION:  PULMONARY EDEMA\n\nIMPRESSION: \n\nIn comparison with the study of ___, there again is substantial enlargement\nof the cardiac silhouette without pulmonary vascular congestion.  This\ndiscordance suggests underlying cardiomyopathy or pericardial effusion.\nThere has been placement of a right IJ Swan-Ganz catheter that extends to the\nright pulmonary artery at the outer limits of the mediastinum.\nNo evidence of acute focal pneumonia.\n', 'EXAMINATION:  CHEST (PORTABLE AP)\n\nINDICATION:  ___ year old man with CHF s/p swan placement now not drawing back \n// evaluate position of swan      evaluate position of swan\n\nCOMPARISON:  Prior chest radiographs since ___ most recently ___.\n\nIMPRESSION: \n\nSwan-Ganz catheter ends in the right descending pulmonary artery at least 2 cm\nbeyond standard placement.  Severe cardiomegaly is chronic.  Pulmonary\nvascular congestion has worsened although there is no pulmonary edema or\nappreciable p

In [75]:
print(train_stays_list[0].get('cxr_metadata'))

[{'dicom_id': '402f6f42-d0f23298-0bc9e9e5-bf1062eb-852e2c56', 'PerformedProcedureStepDescription': 'CHEST (PORTABLE AP)', 'ViewPosition': 'AP', 'ProcedureCodeSequence_CodeMeaning': 'CHEST (PORTABLE AP)', 'ViewCodeSequence_CodeMeaning': 'antero-posterior', 'PatientOrientationCodeSequence_CodeMeaning': 'Erect'}, {'dicom_id': 'a8151392-fe6cfae5-7128b4e0-c8ac92e3-f5c1131d', 'PerformedProcedureStepDescription': 'CHEST (PORTABLE AP)', 'ViewPosition': 'AP', 'ProcedureCodeSequence_CodeMeaning': 'CHEST (PORTABLE AP)', 'ViewCodeSequence_CodeMeaning': 'antero-posterior', 'PatientOrientationCodeSequence_CodeMeaning': 'Erect'}]
